# 0. 라이브러리

In [13]:
import pandas as pd
import numpy as np
import time

import OpenDartReader

In [14]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [15]:
api_key = '462c1cb331fdef6dbc18da2fc8253c2b774e925f'
dart = OpenDartReader(api_key)

- `finstate(corp, bsns_year, reprt_code)`
    - corp: 기업명
    - bsns_year: 사업연도
    - reprt_code: 보고서 유형 11011: 사업보고서, '11012': 반기보고서, '11013': 1분기보고서, '11014: 3분기보고서)
- 이 메서드는 아래 컬럼을 가지는 데이터프레임을 반환합니다.
    - account_nm: 계정명 ex. 자본총계, 영업이익, 당기순이익 등
    - fs_nm: 개별/연결명 ex. 연결재무제표 또는 재무제표
    - sj_nm: 재무제표명 ex. 재무상태표 또는 손익계산서
    - thstrm_dt: 당기일자  ex. 2019.12.31 현재, 2019.01.01 ~ 2019.12.31
    - thstrm_amount: 당기금액 ex. 166,009,000,000
- 계정명과 당기/전기/전전기 금액 컬럼을 바탕으로 우리가 원하는 값을 가져올 수 있습니다

In [16]:
result = dart.finstate("비트나인", 2022, "11011") # 2020년 래몽래인 사업보고서 내 재무 정보
result = result.loc[result.fs_nm == "연결재무제표"] # 연결재무제표 필터링

# account_nm이 당기순이익, 영업이익, 매출액 중 하나인 경우에만 가져오기
result = result.loc[result.account_nm.isin(['당기순이익', '영업이익', '매출액'])]

result = result.loc[:, ['account_nm', 'fs_nm', 'sj_nm', 'thstrm_dt', 'thstrm_amount']]
display(result)

,account_nm,fs_nm,sj_nm,thstrm_dt,thstrm_amount
9,영업이익,연결재무제표,손익계산서,2022.01.01 ~ 2022.12.31,"1,620,879,293"
10,당기순이익,연결재무제표,손익계산서,2022.01.01 ~ 2022.12.31,"-528,858,705"


# 1. 재무변수 수집 함수

In [17]:
def find_financial_ind(corp_nm, yr, inds):
    report = dart.finstate(corp_nm, yr) # 데이터 가져오기

    if report is None:
        # 리포트가 없으면 당기, 전기, 전전기 값 모두 제거
        data = [[corp_nm, yr] + [np.nan] * len(inds)]
        data = [[corp_nm, yr-1] + [np.nan] * len(inds)]
        data = [[corp_nm, yr-2] + [np.nan] * len(inds)]
        return pd.DataFrame(data, columns=['기업명', '연도'] + inds)
    
    else:
        report = report[report.account_nm.isin(inds)]
        if sum(report.fs_nm == '연결재무제표') > 0:
            # 연결재무제표 데이터가 있으면 연결재무제표 사용
            report = report.loc[report.fs_nm == '연결재무제표']

        else:
            # 연결재무제표 데이터가 없으면 일반재무제표 사용
            report = report.loc[report.fs_nm == '재무제표']

        data = []
        for y, c in zip([yr, yr-1, yr-2], ['thstrm_amount', 'frmtrm_amount', 'bfefrmtrm_amount']):
            record = [corp_nm, y]
            for ind in inds:
                # account_nm이 ind인 행의 c 컬럼 값을 가져 옴
                if sum(report.account_nm == ind) > 0:
                    value = report.loc[report.account_nm == ind, c].iloc[0]
                else:
                    value = np.nan

                record.append(value)
            
            data.append(record)

        return pd.DataFrame(data, columns=['기업명', '연도'] + inds)

In [18]:
def str_to_float(value):
    if type(value) == float:
        return value
    elif value == '-':
        return 0
    else:
        return float(value.replace(',', ''))

In [19]:
inds = ['자산총계', '부채총계', '자본총계', '매출액', '영업이익', '당기순이익', '유동자산', '유동부채', '이자비용']
display(find_financial_ind('221610', 2020, inds))

,기업명,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익,유동자산,유동부채,이자비용
0,221610,2020,"118,776,553,482","55,023,990,430","63,752,563,052","16,309,613,271","-8,850,764,284","-13,334,077,895","42,971,947,237","38,296,503,314",NaN
1,221610,2019,"34,679,009,006","9,250,310,220","25,428,698,786","43,888,122,675","422,865,682","674,177,171","24,694,500,349","8,644,404,672",NaN
2,221610,2018,"36,854,979,728","12,081,628,582","24,773,351,146","35,169,316,133","-8,026,556,702","-10,982,140,432","25,839,552,328","11,813,314,409",NaN


# 2. 부도기업 재무 변수 수집

In [20]:
delist_corp = pd.read_csv('./data/부도기업재무실패.csv', dtype={'종목코드':str})
delist_corp.head()

,기업명,종목코드,폐지일,상장일,지속기간
0,자안바이오,221610,2022-01-05,2016-01-27,2170 days
1,럭슬,033600,2021-08-11,2000-11-21,7568 days
2,에스앤씨엔진그룹,900080,2021-06-14,2009-12-04,4210 days
3,미래SCI,028040,2021-04-13,1996-07-27,9026 days
4,바이오빌,065940,2020-07-31,2003-01-07,6415 days


In [21]:
delist_corp['폐지일'] = pd.to_datetime(delist_corp['폐지일'])

delist_codes = delist_corp.종목코드.to_list()
delist_yrs = delist_corp.폐지일.dt.year.to_list()

delist_codes[:3], delist_yrs[:3]

(['221610', '033600', '900080'], [2022, 2021, 2021])

In [22]:
len(delist_codes), len(delist_yrs)

(67, 67)

In [23]:
inds = ['자산총계', '부채총계', '자본총계', '매출액', '영업이익', '당기순이익',\
    '유동부채', '유동자산', '비유동자산', '비유동부채']

delist_data = pd.DataFrame()

for idx, (corp_nm, yr) in enumerate(zip(delist_codes, delist_yrs)):
    print(idx+1, "/", len(delist_codes))
    try:
        yr -= 2
        result = find_financial_ind(corp_nm, yr, inds)
    except:
        pass

    delist_data = pd.concat([delist_data, result], axis=0, ignore_index=True)
    time.sleep(0.5)

for ind in inds:
    delist_data[ind] = delist_data[ind].apply(str_to_float)

1 / 67
2 / 67
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

3 / 67
4 / 67
5 / 67
6 / 67
7 / 67
8 / 67
9 / 67
10 / 67
11 / 67
12 / 67
13 / 67
14 / 67
15 / 67
16 / 67
{'status': '013', 'message': '조회된 데이타가 없습니다.'}
전자공시의 재무데이터는 2015년 이후 데이터를 제공합니다
17 / 67
{'status': '013', 'message': '조회된 데이타가 없습니다.'}
전자공시의 재무데이터는 2015년 이후 데이터를 제공합니다
18 / 67
{'status': '013', 'message': '조회된 데이타가 없습니다.'}
전자공시의 재무데이터는 2015년 이후 데이터를 제공합니다
19 / 67
{'status': '013', 'message': '조회된 데이타가 없습니다.'}
전자공시의 재무데이터는 2015년 이후 데이터를 제공합니다
20 / 67
{'status': '013', 'message': '조회된 데이타가 없습니다.'}
전자공시의 재무데이터는 2015년 이후 데이터를 제공합니다
21 / 67
{'status': '013', 'message': '조회된 데이타가 없습니다.'}
전자공시의 재무데이터는 2015년 이후 데이터를 제공합니다
22 / 67
{'status': '013', 'message': '조회된 데이타가 없습니다.'}
전자공시의 재무데이터는 2015년 이후 데이터를 제공합니다
23 / 67
{'status': '013', 'message': '조회된 데이타가 없습니다.'}
전자공시의 재무데이터는 2015년 이후 데이터를 제공합니다
24 / 67
{'status': '013', 'message': '조회된 데이타가 없습니다.'}
전자공시의 재무데이터는 2015년 이후 데이터를 제공합니다
25 / 67
{'status': '013', 'message': '조회된 데이타가 없습

In [24]:
delist_data_raw = delist_data.copy()
delist_data_raw.to_csv('./data/부도기업재무원본2.csv', index=False)

In [25]:
delist_data.shape

(201, 12)

In [26]:
delist_data.sort_values(by=['기업명', '연도'], inplace=True)
delist_data.drop_duplicates(inplace=True)

In [27]:
delist_data.head(3)

,기업명,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익,유동부채,유동자산,비유동자산,비유동부채
11,028040,2017,5.236055e+10,3.405446e+10,1.830608e+10,1.910892e+10,-4.284786e+09,-6.664955e+09,3.371622e+10,1.456719e+10,1.305708e+10,3.382404e+08
10,028040,2018,6.616323e+10,4.273140e+10,2.343183e+10,5.759349e+10,-4.266505e+09,-2.134382e+10,4.002847e+10,2.963970e+10,1.350099e+10,2.702934e+09
9,028040,2019,5.440533e+10,3.748779e+10,1.691754e+10,1.479797e+10,-5.963843e+09,-2.061870e+10,3.513251e+10,1.182675e+10,1.907107e+10,2.355277e+09


In [28]:
delist_data.isnull().sum()

기업명      0
연도       0
자산총계     0
부채총계     0
자본총계     0
매출액      0
영업이익     0
당기순이익    0
유동부채     0
유동자산     0
비유동자산    0
비유동부채    0
dtype: int64

In [29]:
delist_data.shape

(42, 12)

# 3. 결측치 knn -> 없음

# 4. 재무 비율 변수 구성
- https://gils-lab.tistory.com/38
- https://dacon.io/competitions/official/235946/codeshare/5805
- https://blog.naver.com/o12486vs2/222096044791

### (1) 건전성

In [30]:
# 부채비율 = 총부채 / 총자산 DR (LEV)
delist_data['부채비율'] = delist_data['부채총계'] / delist_data['자본총계'] * 100

# 자기자본비율 DER
delist_data['자기자본비율'] = delist_data['부채총계'] / delist_data['자산총계'] * 100

# 유동부채비율
delist_data['유동부채비율'] = delist_data['유동부채'] / delist_data['자본총계'] * 100

# 유동부채비율
delist_data['비유동부채비율'] = delist_data['비유동부채'] / delist_data['자본총계'] * 100


### (2) 수익성

In [31]:
# 총자산영업이익율 OI/TA
delist_data['총자산영업이익율'] = delist_data['영업이익'] / delist_data['자산총계']

# 총자산순이익율 ROA
delist_data['ROA'] = delist_data['당기순이익'] / delist_data['자산총계']

# 자기자본이익율 ROE
avg_eq = delist_data['자본총계'].rolling(2).mean()
delist_data['ROE'] = delist_data['당기순이익'] / avg_eq
# delist_data.loc[delist_data.연도 == 2020, 'ROE'] = np.nan

# 매출액영업이익율 OPM
delist_data['매출액영업이익율'] = delist_data['영업이익'] / delist_data['매출액']

# 매출액순이익율
delist_data['매출액순이익율'] = delist_data['당기순이익'] / delist_data['매출액']


### (3) 성장성

In [32]:
delist_data['총자산증가율'] = delist_data['자산총계'].diff() / delist_data['자산총계'] * 100
delist_data.loc[delist_data.연도 == 2020, '총자산증가율'] = np.nan

delist_data['매출액증가율'] = delist_data['매출액'].diff() / delist_data['매출액'] * 100
delist_data.loc[delist_data.연도 == 2020, '매출액증가율'] = np.nan

delist_data['당기순이익증가율'] = delist_data['당기순이익'].diff() / delist_data['당기순이익'] * 100
delist_data.loc[delist_data.연도 == 2020, '당기순이익증가율'] = np.nan

delist_data['영업이익증가율'] = delist_data['영업이익'].diff() / delist_data['영업이익'] * 100
delist_data.loc[delist_data.연도 == 2020, '영업이익증가율'] = np.nan

### (4) 유동성

In [33]:
# 유동비율 LIQ
delist_data['유동비율'] = delist_data['유동자산'] / delist_data['유동부채']

### (5) 활동성

In [34]:
# 자산회전율
delist_data['자산회전율'] = delist_data['매출액'] / delist_data['자산총계']

# 부채회전율
delist_data['부채회전율'] = delist_data['매출액'] / delist_data['부채총계']

# 자본회전율
delist_data['자본회전율'] = delist_data['매출액'] / delist_data['자본총계']


### (6) 규모

In [35]:
# 총매출액규모
delist_data['총매출액규모'] = np.log(delist_data['매출액'])

# 총매출액규모
delist_data['총자산규모'] = np.log(delist_data['자산총계'])

/Users/yeorii/opt/anaconda3/envs/yeonii/lib/python3.9/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [36]:
delist_data.head()

,기업명,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익,유동부채,유동자산,비유동자산,비유동부채,부채비율,자기자본비율,유동부채비율,비유동부채비율,총자산영업이익율,ROA,ROE,매출액영업이익율,매출액순이익율,총자산증가율,매출액증가율,당기순이익증가율,영업이익증가율,유동비율,자산회전율,부채회전율,자본회전율,총매출액규모,총자산규모
11,028040,2017,5.236055e+10,3.405446e+10,1.830608e+10,1.910892e+10,-4.284786e+09,-6.664955e+09,3.371622e+10,1.456719e+10,1.305708e+10,3.382404e+08,186.028106,65.038401,184.180412,1.847694,-0.081832,-0.127290,NaN,-0.224230,-0.348788,NaN,NaN,NaN,NaN,0.432053,0.364949,0.561128,1.043856,23.673421,24.681419
10,028040,2018,6.616323e+10,4.273140e+10,2.343183e+10,5.759349e+10,-4.266505e+09,-2.134382e+10,4.002847e+10,2.963970e+10,1.350099e+10,2.702934e+09,182.364761,64.584816,170.829451,11.535310,-0.064485,-0.322593,-1.022754,-0.074080,-0.370594,20.861562,66.821047,68.773372,-0.428486,0.740466,0.870476,1.347802,2.457917,24.776675,24.915391
9,028040,2019,5.440533e+10,3.748779e+10,1.691754e+10,1.479797e+10,-5.963843e+09,-2.061870e+10,3.513251e+10,1.182675e+10,1.907107e+10,2.355277e+09,221.591190,68.904621,207.669089,13.922100,-0.109619,-0.378983,-1.022009,-0.403018,-1.393347,-21.611669,-289.198660,-3.516791,28.460474,0.336633,0.271995,0.394741,0.874711,23.417756,24.719728
32,039790,2014,3.799845e+10,2.077598e+10,1.722247e+10,1.800994e+10,-5.539845e+09,-1.228350e+10,2.016450e+10,2.072301e+10,1.727544e+10,6.114786e+08,120.632972,54.675859,117.082503,3.550470,-0.145791,-0.323263,-0.719596,-0.307599,-0.682040,-43.177772,17.834472,-67.856854,-7.653607,1.027698,0.473965,0.866864,1.045724,23.614190,24.360811
31,039790,2015,5.546998e+10,1.356321e+10,4.190678e+10,1.804536e+10,3.058177e+08,1.193257e+09,1.355177e+10,2.730484e+10,2.816514e+10,1.143408e+07,32.365187,24.451435,32.337903,0.027285,0.005513,0.021512,0.040361,0.016947,0.066125,31.497281,0.196251,1129.409927,1911.485791,2.014854,0.325318,1.330464,0.430607,23.616154,24.739108


In [37]:
delist_data.shape

(42, 31)

In [38]:
delist_data.isnull().sum()

기업명         0
연도          0
자산총계        0
부채총계        0
자본총계        0
매출액         0
영업이익        0
당기순이익       0
유동부채        0
유동자산        0
비유동자산       0
비유동부채       0
부채비율        0
자기자본비율      0
유동부채비율      0
비유동부채비율     0
총자산영업이익율    0
ROA         0
ROE         1
매출액영업이익율    1
매출액순이익율     1
총자산증가율      2
매출액증가율      2
당기순이익증가율    2
영업이익증가율     2
유동비율        0
자산회전율       0
부채회전율       0
자본회전율       0
총매출액규모      0
총자산규모       0
dtype: int64

In [39]:
delist_rows = np.arange(2, len(delist_data), 3)
delist_data = delist_data.iloc[delist_rows, :]

In [40]:
delist_data = delist_data.reset_index().drop(['index'], axis=1)
delist_data.rename(columns={'기업명':'종목코드'}, inplace=True)
delist_data.head()

,종목코드,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익,유동부채,유동자산,비유동자산,비유동부채,부채비율,자기자본비율,유동부채비율,비유동부채비율,총자산영업이익율,ROA,ROE,매출액영업이익율,매출액순이익율,총자산증가율,매출액증가율,당기순이익증가율,영업이익증가율,유동비율,자산회전율,부채회전율,자본회전율,총매출액규모,총자산규모
0,028040,2019,5.440533e+10,3.748779e+10,1.691754e+10,1.479797e+10,-5.963843e+09,-2.061870e+10,3.513251e+10,1.182675e+10,1.907107e+10,2.355277e+09,221.591190,68.904621,207.669089,13.922100,-0.109619,-0.378983,-1.022009,-0.403018,-1.393347,-21.611669,-289.198660,-3.516791,28.460474,0.336633,0.271995,0.394741,0.874711,23.417756,24.719728
1,039790,2016,3.195864e+10,6.558674e+09,2.539997e+10,2.036286e+10,-2.769139e+09,-2.648717e+10,6.538913e+09,1.570197e+10,1.625667e+10,1.976113e+07,25.821584,20.522380,25.743784,0.077800,-0.086648,-0.828795,-0.787059,-0.135990,-1.300759,-73.568036,11.381019,104.505036,111.043785,2.401312,0.637163,3.104722,0.801688,23.736978,24.187708
2,051170,2016,7.747369e+10,6.299852e+10,1.447517e+10,4.348781e+10,-2.192226e+10,-3.706940e+10,5.542319e+10,2.801074e+10,4.946295e+10,7.575326e+09,435.217922,81.316022,382.884669,52.333254,-0.282964,-0.478477,-1.326982,-0.504101,-0.852409,-51.498672,-7.534328,58.751333,44.882294,0.505397,0.561324,0.690299,3.004305,24.495747,25.073204
3,052270,2015,2.522994e+11,2.327077e+11,1.959167e+10,1.923670e+11,-4.868216e+10,-6.327990e+10,7.914426e+10,6.876603e+10,1.835333e+11,1.535634e+11,1187.788920,92.234752,403.968946,783.819974,-0.192954,-0.250813,-1.315163,-0.253069,-0.328954,-33.344395,-52.615191,-62.174918,24.026023,0.868869,0.762456,0.826647,9.818818,25.982671,26.253882
4,058370,2015,3.113357e+10,1.408074e+10,1.705284e+10,1.706374e+10,1.761864e+08,-1.254207e+10,1.365507e+10,1.611290e+10,1.502067e+10,4.256639e+08,82.571215,45.226853,80.075068,2.496147,0.005659,-0.402847,-0.826187,0.010325,-0.735013,35.737690,28.936366,57.214204,1797.058959,1.179994,0.548082,1.211850,1.000639,23.560221,24.161553


In [41]:
delist_data.to_csv('./data/부도기업재무2.csv', index=False)

# 실패한 애들 알아보기

In [42]:
delist_corp = pd.read_csv('./data/부도기업.csv', dtype={'종목코드':str})
print(delist_corp.shape)
delist_corp.head()

(141, 5)


,기업명,종목코드,폐지일,상장일,지속기간
0,코스온,069110,2023-10-20,2003-10-14,7311 days
1,지나인제약,078650,2023-07-21,2010-10-22,4655 days
2,멜파스,096640,2023-07-17,2009-12-18,4959 days
3,엠피씨플러스,050540,2023-05-08,2005-12-12,6356 days
4,제이웨이,058420,2023-04-14,2002-01-10,7764 days


In [43]:
delist_done = pd.read_csv('./data/부도기업재무.csv', dtype={'종목코드':str})
print(delist_done.shape)
delist_done.head()

(74, 31)


,종목코드,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익,유동부채,유동자산,비유동자산,비유동부채,부채비율,자기자본비율,유동부채비율,비유동부채비율,총자산영업이익율,ROA,ROE,매출액영업이익율,매출액순이익율,총자산증가율,매출액증가율,당기순이익증가율,영업이익증가율,유동비율,자산회전율,부채회전율,자본회전율,총매출액규모,총자산규모
0,008800,2020.0,3.199762e+10,1.282887e+10,1.916875e+10,8.087514e+09,-2.933185e+09,-4.336539e+09,1.125048e+10,8.297428e+09,2.370019e+10,1.578396e+09,66.925972,40.093205,58.691757,8.234215,-0.091669,-0.135527,-0.196856,-0.362681,-0.536202,NaN,NaN,NaN,NaN,0.737518,0.252754,0.630415,0.421911,22.813587,24.188927
1,017680,2018.0,6.653430e+10,3.649968e+10,3.003462e+10,5.162126e+10,-6.876169e+09,-1.057761e+10,3.528285e+10,3.706857e+10,2.055660e+10,1.216835e+09,121.525364,54.858442,117.473923,4.051441,-0.103348,-0.158980,-0.359353,-0.133204,-0.204908,15.773120,9.247483,-101.303722,-42.275902,1.050612,0.775859,1.414293,1.718725,24.667199,24.920983
2,023430,2016.0,3.448916e+10,2.518239e+10,9.306768e+09,1.211601e+10,-3.751118e+09,-1.421161e+11,8.108206e+09,6.565781e+09,2.792337e+10,1.707418e+10,270.581460,73.015380,87.121615,183.459844,-0.108762,-4.120602,-4.350440,-0.309600,-11.729612,-85.620040,-3.283316,91.294977,43.884732,0.809770,0.351299,0.481130,1.301849,23.217793,24.263911
3,026260,2017.0,5.171141e+10,2.212413e+10,2.958728e+10,1.198680e+10,-1.174278e+10,-1.283146e+10,1.465260e+10,2.708373e+10,2.462768e+10,7.471526e+09,74.775798,42.783840,49.523307,25.252491,-0.227083,-0.248136,-0.808883,-0.979642,-1.070466,24.280479,-155.490696,-285.724968,-12.145574,1.848391,0.231802,0.541798,0.405133,23.207072,24.668944
4,030270,2019.0,4.586739e+10,2.748884e+10,1.837855e+10,2.550810e+09,-3.967914e+09,-1.452529e+10,1.975572e+10,2.156882e+10,2.379226e+10,7.733123e+09,149.570231,59.931119,107.493338,42.076893,-0.086508,-0.316680,-0.643280,-1.555551,-5.694384,-34.430538,-457.385441,-137.487113,-161.813750,1.091776,0.055613,0.092794,0.138793,21.659677,24.549020


In [44]:
delist_done2 = pd.read_csv('./data/부도기업재무2.csv', dtype={'종목코드':str})
print(delist_done2.shape)
delist_done2.head()

(14, 31)


,종목코드,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익,유동부채,유동자산,비유동자산,비유동부채,부채비율,자기자본비율,유동부채비율,비유동부채비율,총자산영업이익율,ROA,ROE,매출액영업이익율,매출액순이익율,총자산증가율,매출액증가율,당기순이익증가율,영업이익증가율,유동비율,자산회전율,부채회전율,자본회전율,총매출액규모,총자산규모
0,028040,2019,5.440533e+10,3.748779e+10,1.691754e+10,1.479797e+10,-5.963843e+09,-2.061870e+10,3.513251e+10,1.182675e+10,1.907107e+10,2.355277e+09,221.591190,68.904621,207.669089,13.922100,-0.109619,-0.378983,-1.022009,-0.403018,-1.393347,-21.611669,-289.198660,-3.516791,28.460474,0.336633,0.271995,0.394741,0.874711,23.417756,24.719728
1,039790,2016,3.195864e+10,6.558674e+09,2.539997e+10,2.036286e+10,-2.769139e+09,-2.648717e+10,6.538913e+09,1.570197e+10,1.625667e+10,1.976113e+07,25.821584,20.522380,25.743784,0.077800,-0.086648,-0.828795,-0.787059,-0.135990,-1.300759,-73.568036,11.381019,104.505036,111.043785,2.401312,0.637163,3.104722,0.801688,23.736978,24.187708
2,051170,2016,7.747369e+10,6.299852e+10,1.447517e+10,4.348781e+10,-2.192226e+10,-3.706940e+10,5.542319e+10,2.801074e+10,4.946295e+10,7.575326e+09,435.217922,81.316022,382.884669,52.333254,-0.282964,-0.478477,-1.326982,-0.504101,-0.852409,-51.498672,-7.534328,58.751333,44.882294,0.505397,0.561324,0.690299,3.004305,24.495747,25.073204
3,052270,2015,2.522994e+11,2.327077e+11,1.959167e+10,1.923670e+11,-4.868216e+10,-6.327990e+10,7.914426e+10,6.876603e+10,1.835333e+11,1.535634e+11,1187.788920,92.234752,403.968946,783.819974,-0.192954,-0.250813,-1.315163,-0.253069,-0.328954,-33.344395,-52.615191,-62.174918,24.026023,0.868869,0.762456,0.826647,9.818818,25.982671,26.253882
4,058370,2015,3.113357e+10,1.408074e+10,1.705284e+10,1.706374e+10,1.761864e+08,-1.254207e+10,1.365507e+10,1.611290e+10,1.502067e+10,4.256639e+08,82.571215,45.226853,80.075068,2.496147,0.005659,-0.402847,-0.826187,0.010325,-0.735013,35.737690,28.936366,57.214204,1797.058959,1.179994,0.548082,1.211850,1.000639,23.560221,24.161553


In [46]:
delist_corps_done = delist_done.종목코드
delist_corps_done2 = delist_done2.종목코드

delist_corps_fail = delist_corp[~delist_corp.종목코드.isin(delist_corps_done)]
delist_corps_fail = delist_corps_fail[~delist_corps_fail.종목코드.isin(delist_corps_done2)]
print(delist_corps_fail.shape)
delist_corps_fail.head()

(53, 5)


,기업명,종목코드,폐지일,상장일,지속기간
32,럭슬,033600,2021-08-11,2000-11-21,7568 days
87,아이디에스,078780,2016-04-28,2005-11-29,3803 days
90,GT&T,053870,2015-10-15,2001-12-13,5054 days
91,KCW,068060,2015-09-18,2003-01-28,4616 days
92,디아이디,074130,2015-07-23,2005-07-27,3648 days


In [47]:
delist_corps_fail.to_csv('./data/부도기업재무실패2.csv', index=False)